In [1]:
import geojson
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, Point, LineString, shape
from shapely.geometry import mapping
from shapely.ops import transform
import pyproj
import math
import random

In [2]:
# Coordinate transformation setup (WGS84 to UTM)
def get_utm_transformer(lon):
    """Determine UTM zone and return a transformer from WGS84 to UTM."""
    utm_zone = int((lon + 180) / 6) + 1
    utm_crs = f"EPSG:326{utm_zone:02d}"
    transformer = pyproj.Transformer.from_crs("EPSG:4326", utm_crs, always_xy=True)
    return transformer

def transform_geometry(geometry, transformer):
    """Transform GeoJSON geometry coordinates from WGS84 to UTM."""
    return transform(lambda x, y: transformer.transform(x, y), geometry)

# Zone shape generation
def generate_zone_polygon(center_x, center_y, area, shape="rectangle"):
    """Generate a polygon for a zone based on area and shape type (in sqm)."""
    if shape == "rectangle":
        side = math.sqrt(area)
        half_side = side / 2
        coords = [
            (center_x - half_side, center_y - half_side),
            (center_x + half_side, center_y - half_side),
            (center_x + half_side, center_y + half_side),
            (center_x - half_side, center_y + half_side)
        ]
        return Polygon(coords)
    elif shape == "circle":
        radius = math.sqrt(area / math.pi)
        return Point(center_x, center_y).buffer(radius)
    else:
        raise ValueError(f"Unsupported zone shape: {shape}")

# Building shape generation
def generate_building_polygon(center_x, center_y, area, shape="rectangle"):
    """Generate a polygon for a building based on area and shape type (in sqm)."""
    if shape == "rectangle":
        side = math.sqrt(area)
        half_side = side / 2
        coords = [
            (center_x - half_side, center_y - half_side),
            (center_x + half_side, center_y - half_side),
            (center_x + half_side, center_y + half_side),
            (center_x - half_side, center_y + half_side)
        ]
        return Polygon(coords)
    elif shape == "circle":
        radius = math.sqrt(area / math.pi)
        return Point(center_x, center_y).buffer(radius)
    elif shape == "triangle":
        side = math.sqrt((4 * area) / math.sqrt(3))
        height = (math.sqrt(3) / 2) * side
        coords = [
            (center_x, center_y + height / 2),
            (center_x - side / 2, center_y - height / 2),
            (center_x + side / 2, center_y - height / 2)
        ]
        return Polygon(coords)
    else:
        raise ValueError(f"Unsupported building shape: {shape}")

# Convert GeoJSON geometry to Shapely
def geojson_to_shapely(geojson_geom):
    """Convert GeoJSON geometry to Shapely geometry."""
    return shape(geojson_geom)

# Convert Shapely geometry to GeoJSON
def shapely_to_geojson(shapely_geom):
    """Convert Shapely geometry to GeoJSON geometry."""
    return mapping(shapely_geom)

# Validate and fix geometry
def fix_geometry(geom):
    """Validate and fix invalid geometry."""
    if not geom.is_valid:
        return geom.buffer(0)  # Small buffer to fix self-intersections
    return geom

# Check for overlaps and log
def check_no_overlap(zone, no_build_areas):
    """Check if a zone overlaps with no-build areas and log if it does."""
    overlaps = any(zone.intersects(nb) for nb in no_build_areas)
    if overlaps:
        print(f"Warning: Zone overlaps no-build area at coordinates {list(zone.exterior.coords)[:2]}")
    return not overlaps

# Check minimum distance to no-build areas
def check_min_distance_to_no_build(zone, no_build_areas, min_distance):
    """Check if a zone is at least min_distance away from all no-build areas."""
    too_close = any(zone.distance(nb) < min_distance for nb in no_build_areas)
    if too_close:
        print(f"Warning: Zone too close to no-build area at coordinates {list(zone.exterior.coords)[:2]}")
    return not too_close

# Bottom-Left Fill packing for zones with randomization and distance constraints
def pack_zone(building_zone, no_build_areas, area, shape_type, min_zone_distance, min_zone_no_build_distance, placed_zones, randomness=0.1):
    """Pack a zone into the building site using Bottom-Left Fill with randomization and distance constraints."""
    building_zone = fix_geometry(building_zone)
    minx, miny, maxx, maxy = building_zone.bounds
    # Add random offset for starting position
    start_x = minx + random.uniform(-randomness * (maxx - minx), randomness * (maxx - minx))
    start_y = miny + random.uniform(-randomness * (maxy - miny), randomness * (maxy - miny))
    step = math.sqrt(area) / 20  # Small step for smooth sliding
    y = start_y
    while y <= maxy:
        x = start_x
        while x <= maxx:
            zone_poly = generate_zone_polygon(x, y, area, shape_type)
            zone_poly = fix_geometry(zone_poly)
            try:
                if (building_zone.contains(zone_poly) and
                    check_no_overlap(zone_poly, no_build_areas) and
                    check_min_distance_to_no_build(zone_poly, no_build_areas, min_zone_no_build_distance) and
                    all(zone_poly.distance(z[0]) >= min_zone_distance for z in placed_zones)):
                    return zone_poly
            except Exception as e:
                print(f"Geometry check failed at ({x}, {y}): {e}")
            x += step
        y += step
    return None

# Bottom-Left Fill packing for buildings within a zone
def pack_building(zone_poly, no_build_areas, building_area, building_shape, min_distance, placed_buildings):
    """Pack a building into a zone using Bottom-Left Fill."""
    zone_poly = fix_geometry(zone_poly)
    minx, miny, maxx, maxy = zone_poly.bounds
    step = math.sqrt(building_area) / 20  # Small step for smooth sliding
    y = miny
    while y <= maxy:
        x = minx
        while x <= maxx:
            building = generate_building_polygon(x, y, building_area, building_shape)
            building = fix_geometry(building)
            try:
                if (zone_poly.contains(building) and
                    check_no_overlap(building, no_build_areas) and
                    all(building.distance(b[0]) >= min_distance for b in placed_buildings)):
                    return building
            except Exception as e:
                print(f"Building geometry check failed at ({x}, {y}): {e}")
            x += step
        y += step
    return None

# Main planning function with minimum zone-no-build distance
def spatial_planning(
    input_geojson_path,
    urban_density,
    building_config,
    min_zone_distance,
    min_zone_no_build_distance,
    min_building_distance,
    road_width,
    output_geojson_path="output_zones.geojson",
    output_image_path="land_use_plan.png",
    num_plans=1,
    flexibility=0.3
):
    """
    Perform spatial planning using a packing algorithm with a minimum distance to no-build areas.

    Parameters:
    - input_geojson_path (str): Path to input GeoJSON file with site boundaries and constraints.
    - urban_density (float): Percentage of land area available for building (0-100).
    - building_config (dict): Configuration with zone areas, shapes, min counts, and building details.
    - min_zone_distance (float): Minimal distance between zones in meters.
    - min_zone_no_build_distance (float): Minimal distance between zones and no-build areas in meters.
    - min_building_distance (float): Minimal distance between buildings in meters.
    - output_geojson_path (str): Path to save output GeoJSON file with new zones (no buildings).
    - output_image_path (str): Path to save land-use plan image.
    - num_plans (int): Number of land-use plans to generate (default 1).
    - flexibility (float): Factor (0-1) determining how much areas and counts can vary (default 0.3).

    Returns:
    - None (saves GeoJSON and image files).
    """
    # Load and parse GeoJSON, tracking no-build types
    with open(input_geojson_path, 'r') as f:
        input_data = geojson.load(f)

    # Determine UTM zone
    first_lon = input_data.features[0].geometry.coordinates[0][0][0]
    transformer = get_utm_transformer(first_lon)

    # Process input geometries and store no-build types
    building_zones = []
    no_build_areas = []  # List of (geometry, type) tuples
    for feature in input_data.features:
        geom = geojson_to_shapely(feature.geometry)
        geom_utm = transform_geometry(geom, transformer)
        if feature.properties.get("name") == "Зона застройки":
            building_zones.append(geom_utm)
        elif feature.properties.get("restriction") == "no_build":
            if isinstance(geom_utm, LineString):
                geom_utm = geom_utm.buffer(road_width)
            no_build_areas.append((fix_geometry(geom_utm), feature.properties.get("name", "Unknown")))

    # Subtract no-build areas from building zone
    building_zone = fix_geometry(building_zones[0])
    total_area = building_zone.area
    for no_build_geom, _ in no_build_areas:
        if building_zone.intersects(no_build_geom):
            building_zone = fix_geometry(building_zone.difference(no_build_geom))
    max_buildable_area = (urban_density / 100) * total_area

    # Prepare zones to place with randomness and flexibility
    all_plans = []
    for plan_idx in range(num_plans):
        placed_zones = []
        placed_buildings = []
        used_area = 0

        # Generate random zones for this plan
        zones_to_place = []
        for zone_type, config in building_config.items():
            # If public transport is avvailable within building site range do something to alter algorithm's behaviour
            if zone_type == "public_transport":
                continue
            areas = config["areas"]
            min_count = config["min_count"]
            shape = config["shape"]

            # Apply flexibility to areas
            random_areas = []
            for area in areas:
                random_factor = 1 + random.uniform(-flexibility, flexibility)
                adjusted_area = max(1, min(max_buildable_area, area * random_factor))
                random_areas.append(adjusted_area)

            # Apply flexibility to count
            random_count = max(1, int(min_count * (1 + random.uniform(-flexibility, flexibility))))
            random_count = min(random_count, int(max_buildable_area // min(random_areas) if random_areas else 1))

            # Generate list of areas, repeating or extending based on random count
            areas_to_place = random_areas.copy()
            if len(areas_to_place) < random_count and random_areas:
                last_area = random_areas[-1]
                areas_to_place.extend([last_area] * (random_count - len(areas_to_place)))
            zones_to_place.extend((area, zone_type, shape) for area in areas_to_place)
        random.shuffle(zones_to_place)  # Randomize placement order for diversity

        # Pack zones
        for area, zone_type, shape_type in zones_to_place:
            if used_area + area > max_buildable_area:
                print(f"Warning: No space for {zone_type} area of {area} sqm in plan {plan_idx + 1}")
                continue
            zone_poly = pack_zone(building_zone, [geom for geom, _ in no_build_areas], area, shape_type, min_zone_distance, min_zone_no_build_distance, placed_zones, flexibility)
            if zone_poly and check_no_overlap(zone_poly, [geom for geom, _ in no_build_areas]):
                placed_zones.append((zone_poly, zone_type))
                used_area += area

                # Place buildings using Bottom-Left Fill
                if zone_type in ["residential", "commercial"]:
                    config = building_config[zone_type]
                    building_area = config.get("building_area", 0)
                    building_shape = config.get("building_shape", "rectangle")
                    if building_area > 0:
                        max_attempts = 100
                        attempt = 0
                        while attempt < max_attempts:
                            building = pack_building(zone_poly, [geom for geom, _ in no_build_areas], building_area, building_shape, min_building_distance, placed_buildings)
                            if building:
                                placed_buildings.append((building, zone_type))
                                attempt = 0  # Reset attempts on success to keep trying
                            else:
                                attempt += 1
                            # Stop if we've placed a reasonable number of buildings
                            zone_buildings = [b for b, t in placed_buildings if t == zone_type]
                            if len(zone_buildings) >= zone_poly.area // building_area:
                                break

        all_plans.append((placed_zones, placed_buildings))

    # Visualization in a table-like structure with unique no-build colors
    cols = math.ceil(math.sqrt(num_plans))
    rows = math.ceil(num_plans / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

    # Handle single plan case
    if num_plans == 1:
        axes = np.array([axes])  # Convert single Axes to 1D array for consistency
    else:
        axes = axes.flatten()

    # Define unique colors for no-build areas
    no_build_colors = {
        "Лесная зона": "darkred",  # Forest
        "Дорога": "purple",       # Road
        "Unknown": "gray"         # Default for unexpected types
    }

    for idx, (zones, buildings) in enumerate(all_plans):
        if idx >= len(axes):
            break
        ax = axes[idx]
        x, y = building_zone.exterior.xy
        ax.fill(x, y, alpha=0.3, color="green", label="Building Zone")

        # Plot no-build areas with unique colors and labels
        plotted_no_build_labels = set()
        for no_build_geom, no_build_type in no_build_areas:
            x, y = no_build_geom.exterior.xy
            label = f"No-Build Area ({no_build_type})" if no_build_type not in plotted_no_build_labels else ""
            ax.fill(x, y, alpha=0.5, color=no_build_colors.get(no_build_type, "gray"), label=label)
            plotted_no_build_labels.add(no_build_type)

        zone_colors = {"residential": "blue", "commercial": "yellow", "park": "green"}
        plotted_zone_labels = set()
        for zone_geom, zone_type in zones:
            x, y = zone_geom.exterior.xy
            label = zone_type.capitalize() if zone_type not in plotted_zone_labels else ""
            ax.fill(x, y, alpha=0.5, color=zone_colors.get(zone_type, "gray"), label=label)
            plotted_zone_labels.add(zone_type)

        building_colors = {"residential": "darkblue", "commercial": "orange"}
        plotted_building_labels = set()
        for building_geom, zone_type in buildings:
            x, y = building_geom.exterior.xy
            label = f"{zone_type.capitalize()} Building" if zone_type not in plotted_building_labels else ""
            ax.fill(x, y, alpha=0.7, color=building_colors.get(zone_type, "black"), label=label)
            plotted_building_labels.add(zone_type)

        ax.set_title(f"Plan {idx + 1}")
        ax.legend()

    # Hide unused axes if num_plans < rows * cols
    if num_plans > 1:
        for idx in range(num_plans, len(axes)):
            axes[idx].set_visible(False)

    plt.tight_layout()
    plt.savefig(output_image_path)
    plt.close()

    # Save first plan's zones to GeoJSON
    geojson_zones = [geojson.Feature(
        geometry=shapely_to_geojson(zone_geom),
        properties={"type": zone_type}
    ) for zone_geom, zone_type in all_plans[0][0]]
    output_feature_collection = geojson.FeatureCollection(geojson_zones)
    with open(output_geojson_path, 'w') as f:
        geojson.dump(output_feature_collection, f)

    print(f"Generated {num_plans} plan(s). Output saved to {output_geojson_path} and {output_image_path}")

In [ ]:
# Example usage
if __name__ == "__main__":
    input_geojson_fname = "input.geojson"

    config = {
        "residential": {"areas": [300], "shape": "rectangle", "min_count": 2, "building_shape": "rectangle", "building_area": 25},
        "commercial": {"areas": [200], "shape": "circle", "min_count": 1, "building_shape": "rectangle", "building_area": 15},
        "park": {"areas": [300], "shape": "rectangle", "min_count": 1},
        "public_transport": "yes"
    }
    
    spatial_planning(
        input_geojson_path=input_geojson_fname,
        urban_density=80,
        building_config=config,
        min_zone_distance=10,
        min_zone_no_build_distance=0,
        min_building_distance=5,
        road_width=3,
        num_plans=4,
        flexibility=0.5
    )

Generated 4 plan(s). Output saved to output_zones.geojson and land_use_plan.png


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\mazhugich\miniconda3\envs\spatial_urban_planner\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mazhugich\miniconda3\envs\spatial_urban_planner\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\mazhugich\miniconda3\envs\spatial_urban_planner\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Use